In [1]:
import pandas as pd

# List of years to load
years = range(2017, 2025) 

# Load each CSV file and append them to a list
dfs = []
for year in years:
    file_name = f'../github_adv/{year}.csv'
    df = pd.read_csv(file_name)
    dfs.append(df)

# Concatenate all DataFrames into one
df_git = pd.concat(dfs, ignore_index=True)

# Check the combined DataFrame
df_git  # Displays the first few rows of the combined DataFrame


,ID,SOURCE,REVIEWED,PUBLISH_DATE,CWE_IDS,ALIASES,SUMMARY,SEVERITY,ECOSYSTEM,PACKAGE_NAME,VERSION_RANGE,REFERENCES
0,GHSA-8c56-cpmw-89x7,Github,True,2017-12-13T21:38:24Z,CWE-125,CVE-2017-9050,Out-of-bounds read in nokogiri,HIGH,RubyGems,nokogiri,"[0,1.8.1)",https://nvd.nist.gov/vuln/detail/CVE-2017-9050...
1,GHSA-gj4p-3wh3-2rmf,Github,True,2017-12-21T00:47:25Z,CWE-22,CVE-2017-17042,Arbitrary file read vulnerability in yard server,HIGH,RubyGems,yard,"[0,0.9.11)",https://nvd.nist.gov/vuln/detail/CVE-2017-1704...
2,GHSA-h9mj-fghc-664w,Github,True,2017-12-28T22:51:58Z,CWE-674,CVE-2017-10910,Denial of Service in mqtt,MODERATE,npm,mqtt,"[2.0.0,2.15.0)",https://nvd.nist.gov/vuln/detail/CVE-2017-1091...
3,GHSA-pm9p-9926-w68m,Github,True,2017-12-28T22:52:47Z,CWE-400,CVE-2016-10703,Denial of Service in ecstatic,HIGH,npm,ecstatic,"[0,2.0.0)",https://nvd.nist.gov/vuln/detail/CVE-2016-1070...
4,GHSA-pv8x-p9hq-j328,Github,True,2017-12-18T22:27:53Z,CWE-88,CVE-2017-1001003,Arbitrary Code Execution in mathjs,CRITICAL,npm,mathjs,"[0,3.17.0)",https://nvd.nist.gov/vuln/detail/CVE-2017-1001...
...,...,...,...,...,...,...,...,...,...,...,...,...
20963,GHSA-29w6-c52g-m8jc,Github,True,2024-01-31T18:05:46Z,NaN,NaN,C5 Firefly III CSV Injection.,MODERATE,Packagist,grumpydictator/firefly-iii,"[0,6.1.7)",https://github.com/firefly-iii/firefly-iii/sec...
20964,GHSA-2cvg-w29m-j8xc,Github,True,2024-01-24T21:30:33Z,CWE-94,CVE-2023-24676,Arbitrary Code Execution in Processwire,HIGH,Packagist,processwire/processwire,"[0,3.0.210]",https://nvd.nist.gov/vuln/detail/CVE-2023-2467...
20965,GHSA-2jxw-4hm4-6w87,Github,True,2024-01-22T03:30:26Z,CWE-89,CVE-2024-23751,SQL injection in llama-index,CRITICAL,PyPI,llama-index,"[0,0.9.35]",https://nvd.nist.gov/vuln/detail/CVE-2024-2375...
20966,GHSA-2mqj-m65w-jghx,Github,True,2024-01-10T15:46:00Z,CWE-426,CVE-2024-22190,Untrusted search path under some conditions on...,HIGH,PyPI,GitPython,"[0,3.1.41)",https://github.com/gitpython-developers/GitPyt...


In [3]:
import ast

refs = df_git['REFERENCES'].tolist()
ids = df_git['ID'].tolist()

git_links = {}

for i, r in enumerate(refs):
    # print(r)
    r_ = r.split(', ')
    # print(r_)
    package_links = set()
    for item in r_:
        if 'https://github.com/' in item and 'https://github.com/advisories' not in item:
            args = item.split('/')
            if len(args) == 5:
                package_links.add(item)
            elif len(args) > 5:
                url = 'https://github.com/' + args[3] + '/' + args[4]
                package_links.add(url)
        # if item['type'] == 'PACKAGE':
            # git_links[ids[i]] = [item['url']]
    git_links[ids[i]] = package_links

print(len(git_links), len(ids))
            

20968 20968


In [4]:
import re

# Function to check for GitHub URL based on various conditions
def match_github_link(package_name, link):
    # Convert both to lowercase to make matching case-insensitive
    package_name = package_name.lower()
    link = link.lower()

    # Condition 1: Exact match
    if package_name in link:
        return True

    # Condition 2: Replace dots with dashes and check if part of the link
    if package_name.replace('.', '-') in link:
        return True

    # Condition 3: Check if the package name is a substring of the link (loose match)
    if link.split('/')[-1] in package_name:
        return True
    
    if link.split('/')[-1] in package_name.replace('.', '-'):
        return True

    return False

# Function to find GitHub URL for each row based on matching conditions
def find_github_link(row):
    vuln_id = row['ID']
    package_name = row['PACKAGE_NAME']
    
    # Get GitHub links for this vulnerability ID
    github_links = git_links.get(vuln_id, [])
    
    # Check if any link matches the package name based on custom conditions
    for link in github_links:
        if match_github_link(package_name, link):
            return link
            break
    return None

# Add a new column 'GitHub URL' by applying the function to each row
df_git['GitHub URL'] = df_git.apply(find_github_link, axis=1)

# Display the updated DataFrame
df_git.head()

,ID,SOURCE,REVIEWED,PUBLISH_DATE,CWE_IDS,ALIASES,SUMMARY,SEVERITY,ECOSYSTEM,PACKAGE_NAME,VERSION_RANGE,REFERENCES,GitHub URL
0,GHSA-8c56-cpmw-89x7,Github,True,2017-12-13T21:38:24Z,CWE-125,CVE-2017-9050,Out-of-bounds read in nokogiri,HIGH,RubyGems,nokogiri,"[0,1.8.1)",https://nvd.nist.gov/vuln/detail/CVE-2017-9050...,None
1,GHSA-gj4p-3wh3-2rmf,Github,True,2017-12-21T00:47:25Z,CWE-22,CVE-2017-17042,Arbitrary file read vulnerability in yard server,HIGH,RubyGems,yard,"[0,0.9.11)",https://nvd.nist.gov/vuln/detail/CVE-2017-1704...,https://github.com/lsegal/yard
2,GHSA-h9mj-fghc-664w,Github,True,2017-12-28T22:51:58Z,CWE-674,CVE-2017-10910,Denial of Service in mqtt,MODERATE,npm,mqtt,"[2.0.0,2.15.0)",https://nvd.nist.gov/vuln/detail/CVE-2017-1091...,https://github.com/mqttjs/MQTT.js
3,GHSA-pm9p-9926-w68m,Github,True,2017-12-28T22:52:47Z,CWE-400,CVE-2016-10703,Denial of Service in ecstatic,HIGH,npm,ecstatic,"[0,2.0.0)",https://nvd.nist.gov/vuln/detail/CVE-2016-1070...,https://github.com/jfhbrook/node-ecstatic
4,GHSA-pv8x-p9hq-j328,Github,True,2017-12-18T22:27:53Z,CWE-88,CVE-2017-1001003,Arbitrary Code Execution in mathjs,CRITICAL,npm,mathjs,"[0,3.17.0)",https://nvd.nist.gov/vuln/detail/CVE-2017-1001...,https://github.com/josdejong/mathjs


In [7]:
df_git.to_csv(f'../data/advisory_crawl_with_git.csv', index=False)